In [1]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(208)

### Graph implementation and related core graph algorithms

In [2]:
######################################################################################################
# Implementation of n-ary trees (for use in generating random trees to test various DP algorithms on)
######################################################################################################


# Generic tree node class
class TreeNode:
    def __init__(self, weight=None, parent=None, depth=None, size=None):
        # children of this node in tree
        self.children = []
        # weight of edge from parent to this node
        self.weight = weight
        # parent of this node in tree
        self.parent = parent
        # depth of this node in overall tree
        self.depth = depth
        # size of subtree rooted at this node
        self.size = size
        # heaviest child of this node
        self.heavy_child = None

        
# Generates a complete n-ary tree with random integer weights in [0, B] and V vertices
# Returns root node of generated tree
def generate_complete_tree(n, B, V, depth=0, first_root=True):
    if V == 1:
        return TreeNode(weight=np.random.randint(0, B + 1), depth=depth, size=1)

    if first_root:
        curr = TreeNode(weight=0, depth=depth, size=V)
    else:
        curr = TreeNode(weight=np.random.randint(0, B + 1), depth=depth, size=V)
    
    # have V-1 > 0 vertices to split among n subtrees
    left, j, each = V - 1, 0, 0
    while left > n ** (j + 1):
        each += n ** j
        left -= n ** (j + 1)
        j += 1
    
    for i in range(n):
        newV = each + min(left, n ** j)
        left -= min(left, n ** j)
        
        if newV > 0:
            child = generate_complete_tree(n, B, newV, depth + 1, first_root=False)
            child.parent = curr
            curr.children.append(child)
    
    return curr

# Generates a degenerate tree that is a path graph with random integer weights 
# in [0, B] and V vertices. Returns root node of the generated tree
def generate_path_graph(n, B, V):
    depth = 0
    ptr = root = TreeNode(weight=0, depth=depth, size=V)
    V -= 1
    while V > 0:
        depth += 1
        child = TreeNode(weight=np.random.randint(0, B + 1), depth=depth, size=V)
        ptr.children = [child]
        child.parent = ptr
        ptr = child
        V -= 1
    return root


# Prints out level-by-level traversal of subtree rooted at node root
def level_traversal(root):
    levels = []
    
    def traverse(node):
        if not node: return
        if len(levels) <= node.depth:
            levels.append([])
        levels[node.depth].append((node.weight, node.parent.weight if node.parent else None, node.size))
        for child in node.children:
            traverse(child)
            
    traverse(root)
    print(levels)

In [3]:
# Returns lowest common ancestor of u and v
# Basic implementation of LCA for simplicity (i.e., not the most efficient)
def LCA(u, v):
    if u.depth < v.depth:
        u, v = v, u
    while u.depth > v.depth:
        u = u.parent
    while u != v:
        u, v = u.parent, v.parent
    return u

### True all-pairs distances function and related algorithms

In [4]:
# Compute true distances from all vertices to root v0
def dfs_true_distances_root(node, d, curr_dist=0):
    if not node: return
    curr_dist += node.weight
    d[node] = curr_dist
    for child in node.children:
        dfs_true_distances_root(child, d, curr_dist)
    curr_dist -= node.weight

# Compute true distances between all vertices and root in tree
def true_distances_root(v0):
    d = dict()
    dfs_true_distances_root(v0, d)
    return d

# Computes distances between all pairs of vertices in tree given distances between all vertices and root
def distances_all_pairs(v0, d_root):
    # Use LCA to compute distances between all pairs of vertices
    dd = dict()
    nodes = list(d_root.keys())
    for i in range(len(nodes)):
        for j in range(i, len(nodes)):
            dd[(nodes[j], nodes[i])] = dd[(nodes[i], nodes[j])] = d_root[nodes[i]] + d_root[nodes[j]] - 2 * d_root[LCA(nodes[i], nodes[j])]
        
    return dd

# Compute true distances between all pairs of vertices in tree
def true_distances_all_pairs(v0):
    return distances_all_pairs(v0, true_distances_root(v0))
    

### Basic composition for private all-pairs distances

In [5]:
######################################################################################################
# Naive Algorithm 1: Basic composition for all-pairs distances in trees
######################################################################################################

def basic_distances_all_pairs(v0, V, epsilon):
    d = true_distances_all_pairs(v0)
    # Add appropriate Laplacian noise to all distances computed, and release
    for pair in d:
        d[pair] += np.random.laplace(loc=0, scale=V*V/epsilon)
    return d

### Synthetic graph for private all-pairs distances

In [6]:
######################################################################################################
# Naive Algorithm 2: generate synthetic tree and return all-pairs distances on that
######################################################################################################

def dfs_synthetic_distances_all_pairs(node, prev, epsilon, synthetic, first_root=True):
    if not node: return
    if not first_root:
        if synthetic:
            prev[node] = node.weight
            node.weight += np.random.laplace(loc=0, scale=1/epsilon)
        else:
            node.weight = prev[node]
    for child in node.children:
        dfs_synthetic_distances_all_pairs(child, prev, epsilon, synthetic, first_root=False)

        
def synthetic_distances_all_pairs(v0, V, epsilon):
    prev = dict()
    # Add Lap(1/epsilon) noise to each edge in tree, to release epsilon-DP synthetic tree
    dfs_synthetic_distances_all_pairs(v0, prev, epsilon, synthetic=True)
    # Compute true all-pairs distances on synthetic tree
    d = true_distances_all_pairs(v0)
    # Undo noise added to each edge 
    dfs_synthetic_distances_all_pairs(v0, prev, epsilon, synthetic=False)
    return d

### Sealfon Algorithm 1 for private all-pairs distances

In [7]:
######################################################################################################
# Sealfon Algorithm 1: for all-pairs distances in trees
######################################################################################################

# computes private distances between all vertices and root v0
def alg1(v0, V, epsilon):
    if V == 1: 
        return {v0: 0}

    # find v* = ptr (splitting vertex of recursion)
    path, ptr, pathlength = [], v0, 0
    while not all([child.size <= V / 2 for child in ptr.children]):
        for child in ptr.children:
            if child.size > V / 2:
                path.append(ptr)
                ptr = child
                pathlength += ptr.weight
                break

    assert (ptr.size > V / 2 and all([child.size <= V / 2 for child in ptr.children]))

    d = dict()
    d[ptr] = pathlength + np.random.laplace(loc=0, scale=np.log2(V) / epsilon)
    d[v0] = 0
    for child in ptr.children:
        d[child] = d[ptr] + child.weight + np.random.laplace(loc=0, scale=np.log2(V) / epsilon)

    # cut off subtree containing ptr to prepare for recursion
    old_ptr_children = ptr.children
    old_ptr_size = ptr.size
    ptr.children = []
    ptr.size = 1
    for node in path:
        node.size -= old_ptr_size - 1

    # recursively compute distances to roots in subtrees
    ds = [alg1(v, v.size, epsilon) for v in old_ptr_children]
    for i in range(len(ds)):
        for v in ds[i]:
            d[v] = d[old_ptr_children[i]] + ds[i][v]

    d0 = alg1(v0, v0.size, epsilon)
    for v in d0:
        d[v] = d[v0] + d0[v]

    # glue subtree that we cut back onto the tree, so as to not modify input tree
    ptr.children = old_ptr_children
    ptr.size = old_ptr_size
    for node in path:
        node.size += old_ptr_size - 1

    # return private distances between all vertices and root v0
    return d

    
def sealfon_distances_all_pairs(v0, V, epsilon):
    # use above computed distances to root v0 to compute all-pairs distances
    return distances_all_pairs(v0, alg1(v0, V, epsilon))

### Fan and Li Algorithms 1/2 for private all-pairs distances

In [8]:
######################################################################################################
# Fan and Li Algorithms 1 + 2: for heavy path decomposition used for all-pairs distances in trees
######################################################################################################

def tree_decomposition(T):
    # list of roots of subtrees (whose edge from their parent isn't heavy)
    R = []
    # list of nodes that are the heads of heavy paths in the tree T
    S = [T]
    ptr = T
    while ptr.children:
        maxChildSize = max([child.size for child in ptr.children])
        heavy_candidates = [child for child in ptr.children if child.size == maxChildSize]
        ptr.heavy_child = np.random.choice(heavy_candidates)
        
        for child in ptr.children:
            if child is not ptr.heavy_child:
                R.append(child)
        ptr = ptr.heavy_child
        
    for v in R:
        S += tree_decomposition(v)
    return S
    

def dfs_fanli_distances_all_pairs(node, nodes, prev, epsilon, add_noise=True):
    if not node: return
    nodes.append(node)
    for child in node.children:
        if child is not node.heavy_child:
            if add_noise:
                prev[child] = child.weight
                child.weight += np.random.laplace(loc=0, scale=1/epsilon)
            else:
                child.weight = prev[child]
    for child in node.children:
        dfs_fanli_distances_all_pairs(child, nodes, prev, epsilon, add_noise)
        

# returns the path length from u up to LCA (of u with some other vertex v)
def path_length(u, dd, lca):
    dist = 0
    ptr = u
    while ptr is not lca:
        start = ptr
        while ptr is not lca and ptr.parent and ptr.parent.heavy_child is ptr:
            ptr = ptr.parent
        dist += dd[(start, ptr)]
        if ptr is not lca and ptr.parent:
            dist += ptr.weight
            ptr = ptr.parent
    return dist 


def fanli_distances_all_pairs(T, V, epsilon):
    dd = dict()
    # list of heads of heavy paths in T
    S = tree_decomposition(T)
    for head in S:
        # isolate heavy path from rest of tree, to pass into sealfon_distances_all_pairs 
        # to compute all-pairs distances within this heavy path only
        new_size = 1
        old_sizes, old_children = [], []
        
        ptr = head
        while ptr.heavy_child:
            old_children.append(ptr.children)
            old_sizes.append(ptr.size)
            ptr.children = [ptr.heavy_child]
            ptr = ptr.heavy_child
            new_size += 1
        
        ptr = head
        while ptr.heavy_child:
            ptr.size = new_size
            ptr = ptr.heavy_child
            new_size -= 1
        
        d = sealfon_distances_all_pairs(head, head.size, epsilon)
        dd = {**dd, **d}
        
        # glue heavy path back onto rest of tree, so as to not modify input tree
        while ptr.parent and ptr.parent.heavy_child is ptr:
            ptr = ptr.parent
            ptr.size = old_sizes.pop()
            ptr.children = old_children.pop()
    
    nodes = []
    prev = dict()
    
    # add noise to light edges
    dfs_fanli_distances_all_pairs(T, nodes, prev, epsilon, add_noise=True)      
    
    # compute remaining all-pairs distances of nodes involving multiple heavy paths
    for i in range(len(nodes) - 1):
        for j in range(i + 1, len(nodes)):
            # if distance between these 2 nodes isn't already computed within heavy paths, compute it
            if (nodes[i], nodes[j]) not in dd:
                lca = LCA(nodes[i], nodes[j])      
                # add paths from nodes[i] up to lca and from nodes[j] up to lca
                dd[(nodes[i], nodes[j])] = dd[(nodes[j], nodes[i])] = path_length(nodes[i], dd, lca) + path_length(nodes[j], dd, lca)
                
    # remove noise from light edges, so as to not modify input tree
    dfs_fanli_distances_all_pairs(T, nodes, prev, epsilon, add_noise=False)
    
    return dd

### Example tree and application of all-pairs distances algorithms

In [9]:
node0 = TreeNode(weight=0, size=9, depth=0)
node1 = TreeNode(weight=1, parent=node0, size=3, depth=1)
node2 = TreeNode(weight=2, parent=node1, size=2, depth=2)
node3 = TreeNode(weight=3, parent=node2, size=1, depth=3)
node4 = TreeNode(weight=4, parent=node0, size=4, depth=1)
node5 = TreeNode(weight=5, parent=node0, size=1, depth=1)
node6 = TreeNode(weight=6, parent=node4, size=2, depth=2)
node7 = TreeNode(weight=7, parent=node4, size=1, depth=2)
node8 = TreeNode(weight=8, parent=node6, size=1, depth=3)

node0.children = [node1, node4, node5]
node1.children = [node2]
node2.children = [node3]
node4.children = [node6, node7]
node6.children = [node8]

In [10]:
# prints level-by-level in tree, from the root downwards, (node.weight, node.parent.weight, node.size)
level_traversal(node0)

[[(0, None, 9)], [(1, 0, 3), (4, 0, 4), (5, 0, 1)], [(2, 1, 2), (6, 4, 2), (7, 4, 1)], [(3, 2, 1), (8, 6, 1)]]


In [11]:
d = true_distances_all_pairs(node0)
print(d[(node3, node5)])

11


In [12]:
d = basic_distances_all_pairs(node0, node0.size, epsilon=1)
print(d[(node3, node5)])

14.012106116823535


In [13]:
d = synthetic_distances_all_pairs(node0, node0.size, epsilon=1)
print(d[(node3, node5)])

11.0421586063295


In [14]:
d = sealfon_distances_all_pairs(node0, node0.size, epsilon=1)
print(d[(node3, node5)])

9.829592219262501


In [15]:
d = fanli_distances_all_pairs(node0, node0.size, epsilon=1)
print(d[(node3, node5)])

12.413415161902327


### Analysis and comparison of private all-pairs distances algorithms

In [16]:
dp = {0: basic_distances_all_pairs, 1: synthetic_distances_all_pairs, 2: sealfon_distances_all_pairs, 3: fanli_distances_all_pairs}
names = {basic_distances_all_pairs: "Basic", synthetic_distances_all_pairs: "Synthetic", sealfon_distances_all_pairs: "Sealfon", fanli_distances_all_pairs: "Fan and Li"}
trials = 3

def compare_and_plot(n, B, Vs, epsilon, generate):
    # average maximum amount an edge weight differs between DP answer and true answer
    max_errors = [[] for _ in range(2)]

    for V in Vs:
        if V % 100 == 0:
            print(V)
        max_errors_trial = [[float('-inf') for _ in range(trials)] for _ in range(2)]

        for i in range(trials):
            t = generate(n, B, V)
            true_d = true_distances_all_pairs(t)
            DP_d = []
            for j in range(1, 3):
                DP_d.append(dp[j](t, t.size, epsilon=epsilon))

            for (u, v) in true_d:
                for j in range(1, 3):
                    max_errors_trial[j-1][i] = max(max_errors_trial[j-1][i], abs(true_d[(u, v)] - DP_d[j-1][(u, v)]))

        for i in range(2):
            max_errors_trial[i] = sum(max_errors_trial[i])/len(max_errors_trial[i])
            max_errors[i].append(max_errors_trial[i])
    
    for i in range(1, 3):
        plt.plot(Vs, max_errors[i-1], label=names[dp[i]])
    plt.xlabel("Number of Vertices")
    plt.ylabel("Average Max Error")
    plt.legend()
    plt.show()
        

In [17]:
# # Base versions
# compare_and_plot(n=2, B=1000, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)
# compare_and_plot(n=4, B=1000, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)

# # Varying epsilon
# compare_and_plot(n=2, B=1000, Vs=range(1, 501), epsilon=0.1, generate=generate_complete_tree)
# compare_and_plot(n=2, B=1000, Vs=range(1, 501), epsilon=10, generate=generate_complete_tree)

# # Varying n-ary tree
# compare_and_plot(n=2, B=1000, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)
# compare_and_plot(n=8, B=1000, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)
# compare_and_plot(n=12, B=1000, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)

# # Varying weight bounds B
# compare_and_plot(n=4, B=100, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)
# compare_and_plot(n=4, B=10000, Vs=range(1, 501), epsilon=1, generate=generate_complete_tree)

# # Testing path graphs
# compare_and_plot(n=None, B=1000, Vs=range(1, 501), epsilon=1, generate=generate_path_graph)

In [ ]:
# Try larger path graphs
compare_and_plot(n=None, B=1000, Vs=range(1, 1001), epsilon=1, generate=generate_path_graph)

100
200
300
400
500
600
700
800
